# import

In [58]:
import cv2
import os
import os.path as osp
from glob import glob
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video
from util import load_json, save_json
from copy import copy

In [2]:
import time

# path

In [3]:
data_name = 'player'
input_dir = osp.join('../input', data_name)
data_dir = osp.join('../output', data_name, 'tracking')
label_save_dir = osp.join('../output', data_name, 'label')
reid_dir = osp.join('../output', data_name, 'reid')
track_label_path = osp.join('../output/')
# movie_dir = osp.join('../output', data_name, 'movie')
movie_dir = osp.join('../output', data_name, 'movie_process')
mot_pid_dir = osp.join('../output', data_name, 'mot_pid')
mot_process_save_dir = osp.join('../output', data_name, 'mot_pid_process')

# function

In [4]:
def listdirs(dir_path, name_condition=None):
    dir_names = os.listdir(dir_path)
    if name_condition is not None:
        dir_names = [dir_name for dir_name in dir_names if dir_name.startswith(name_condition)]
    dir_names.sort()
    return dir_names

In [5]:
def save_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

def load_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

In [6]:
def draw_bbox(img, bboxes, c=(0,255,0)):
    for bbox in bboxes:
        cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), 
                      (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), 
                      (0,255,0), 2, lineType=cv2.LINE_AA)
        ct = [bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2]
        txt = '{}'.format(bbox[4])
        cv2.putText(img, txt, (int(ct[0]), int(ct[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    (0, 0, 0), thickness=3, lineType=cv2.LINE_AA)
        cv2.putText(img, txt, (int(ct[0]), int(ct[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    c, thickness=2, lineType=cv2.LINE_AA)

# all

In [109]:
games = ['game1']
games = None

clips = ['Clip1']
clips = None

In [110]:
template_org = {'fillColor': [255, 0, 0, 128],
 'imageData': None,
 'imageHeight': 720,
 'imagePath': '',
 'imageWidth': 1280,
 'lineColor': [0, 255, 0, 128],
 'shapes': [
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': 'person',
   'line_color': None,
   'points': [[None, None],[None, None]],
   'shape_type': 'rectangle'},
  ],
 'version': '4.2.10'}

In [115]:
if games is None:
    games = listdirs(input_dir, 'game')
    
columns = ['frame_index', 'tracking_id', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'image_name']
for game in tqdm(games):
    print(game)
    game_dir = osp.join(input_dir, game)
    if clips is None:
        clips = listdirs(game_dir, 'Clip')
    
    track_pid_paths = glob(osp.join(reid_dir, game, '*.csv'))
    
    for clip in clips:
        print(clip)
        mot_pid_process_save_path = osp.join(mot_process_save_dir, game, '{}.csv'.format(clip))
        mot_pid_df = pd.read_csv(mot_pid_process_save_path)
        
        frame_indexes = mot_pid_df.frame_index.unique()
        for frame_index in frame_indexes:
            image_name = '{:04}.jpg'.format(frame_index)
            label_name = '{:04}.json'.format(frame_index)
            
            frame_mot_df = mot_pid_df[mot_pid_df.frame_index==frame_index]
            template = copy(template_org)
            template['imagePath'] = image_name
            shapes = []
            for idx, item in frame_mot_df.iterrows():
                
                template_shape = copy(template_org['shapes'][0])
                xmin = item.xmin
                ymin = item.ymin
                xmax = item.xmax
                ymax = item.ymax
                person_name = item.person_name

                template_shape['points'] = [[xmin, ymin],[xmax,ymax]]

                template_shape['label'] = 'person'
                template_shape['person_name'] = person_name
                shapes.append(template_shape)

            template['shapes'] = shapes
            label_save_path = osp.join(input_dir, game, clip, label_name)
            
            if osp.exists(label_save_path):
                os.remove(label_save_path)
            save_json(template, label_save_path)
                
            # 0000.jpg is not labeled use 0001 label
            if frame_index == 1:
                image_name = '{:04}.jpg'.format(0)
                label_name = '{:04}.json'.format(0)
                label_save_path = osp.join(input_dir, game, clip, '0000.json')
                if osp.exists(label_save_path):
                    os.remove(label_save_path)
                template['imagePath'] = '0000.jpg'
                save_json(template, label_save_path)
                
            
    clips = None

  0%|          | 0/10 [00:00<?, ?it/s]

game1
Clip1
Clip10
Clip11
Clip12
Clip13
Clip2
Clip3
Clip4
Clip5
Clip6
Clip7
Clip8
Clip9


 10%|█         | 1/10 [00:02<00:25,  2.84s/it]

game10
Clip1
Clip10
Clip11
Clip12
Clip2
Clip3
Clip4
Clip5
Clip6
Clip7


 20%|██        | 2/10 [00:05<00:23,  2.92s/it]

...

 80%|████████  | 8/10 [00:23<00:05,  2.79s/it]

Clip9
game8
Clip1
Clip2
Clip3
Clip4
Clip5
Clip6
Clip7
Clip8
Clip9


 90%|█████████ | 9/10 [00:26<00:02,  2.80s/it]

game9
Clip1
Clip2
Clip3
Clip4
Clip5
Clip6
Clip7
Clip8
Clip9


100%|██████████| 10/10 [00:28<00:00,  2.83s/it]
